### All needed imports

In [1]:
import os
import cv2
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


/home/sylvia/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Limit GPU Usage

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

2023-08-01 12:02:48.171695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.207432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.207589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.209138: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

1 Physical GPUs, 1 Logical GPUs


2023-08-01 12:02:48.209396: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.209528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.209636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.598373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 12:02:48.598515: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

### Preprocessing Data for Model 1

#### Video Paths

In [3]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

#### Processing code, returns frame (224,224,3) and frame_diffs

In [4]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
            
            
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [5]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    return combined_input

In [6]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)

# checking data of still_diff
# for i, x in enumerate(still_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)

# checking data of still_diff
# for i, x in enumerate(falling_diff):
#     print(" i= "+str(i)+" mean = "+str(np.mean(x)))

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)

# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]


# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

(1080, 224, 224, 3)
(1080, 224, 224)
(1080,)


#### Create the model

In [7]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Convolutional layers for image processing
    x = Conv2D(32, (3, 3), activation='relu')(combined_input)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model

In [8]:
model = create_model()
model.summary()
model.fit(dataset, validation_data=val_dataset, epochs=10)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856 

Epoch 1/10


2023-08-01 12:03:15.376163: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8600
2023-08-01 12:03:15.762034: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


68/68 [==============================] - 4s 42ms/step - loss: 4.9577 - accuracy: 0.8685 - val_loss: 0.0156 - val_accuracy: 1.0000
Epoch 2/10
68/68 [==============================] - 2s 36ms/step - loss: 0.1581 - accuracy: 0.9667 - val_loss: 0.1022 - val_accuracy: 0.9537
Epoch 3/10
68/68 [==============================] - 3s 37ms/step - loss: 0.0609 - accuracy: 0.9787 - val_loss: 0.0178 - val_accuracy: 0.9907
Epoch 4/10
68/68 [==============================] - 3s 40ms/step - loss: 0.0324 - accuracy: 0.9880 - val_loss: 0.0065 - val_accuracy: 0.9954
Epoch 5/10
68/68 [==============================] - 3s 38ms/step - loss: 0.0148 - accuracy: 0.9981 - val_loss: 0.0096 - val_accuracy: 0.9954
Epoch 6/10
68/68 [==============================] - 2s 36ms/step - loss: 0.0354 - accuracy: 0.9926 - val_loss: 0.7457 - val_accuracy: 0.8704
Epoch 7/10
68/68 [==============================] - 3s 38ms/step - loss: 0.0863 - accuracy: 0.9861 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 8/10
68/68 [======